In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
#helper function
def imshow(img):
    plt.figure(figsize=(10,7))
    plt.imshow(img, cmap='gray')

In [7]:
#the video must be in the same folder as this file

def lane_detecter(input_video):
    
    cap = cv2.VideoCapture(input_video) 
    
    #Intial checkup for video
    if not cap.isOpened():
        print("Cannot open video")
        exit()
    while True:
        
        ret, frame = cap.read()
        
        if not ret:
            print("Can't receive frame. Exiting ...")
            break
        blurred = cv2.GaussianBlur(frame, (3, 3), 1)
        gray_image = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)
        inv_gray_image = ~gray_image

        thresh_hold_image = cv2.adaptiveThreshold(inv_gray_image, 200, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 7, 10)
        closed = cv2.morphologyEx(thresh_hold_image, cv2.MORPH_OPEN, (3,3))

        dilated = cv2.dilate(closed, (3,3), iterations=1) 
        cannyed = cv2.Canny(dilated, 30, 100)
        
        #cannyed_crop = cannyed[:, 280:750]
        
        height, width = cannyed.shape[0], cannyed.shape[1]

        roi_vert = [(0, height),(width/2.1, height/2.1),(width, height)]
        
        def roi_mask(img, points):
            mask = np.zeros_like(img)
            match_mask_color = 255
            cv2.fillPoly(mask, points, match_mask_color)
            masked_image = cv2.bitwise_and(img, mask)
            return masked_image
        
        
        roi_image = roi_mask(cannyed, np.array([roi_vert], np.int32))
        
        lines = cv2.HoughLinesP(roi_image, rho = 1, theta = np.pi / 60, threshold = 21, 
                                lines = np.array([]), minLineLength = 21, maxLineGap = 233)
        
        
        img_copy = frame.copy()
        line_image = np.zeros((img_copy.shape[0], img_copy.shape[1], img_copy.shape[2]), dtype=np.uint8)
    
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 5)
    
        img_copy = cv2.addWeighted(img_copy, 0.8,line_image, 1, 0)
        
        
        cv2.imshow('lane_detector', img_copy)
        #cv2.imshow('lane_detector', roi_image)
        
        cv2.waitKey(30)
        if cv2.waitKey(1) == ord('q'):
            break
            
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()
    
    
lane_detecter('solidWhiteRight.mp4')